<a href="https://colab.research.google.com/github/Anu-jo/G2M_cab/blob/main/week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install pandas dask modin[ray] ray pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [ ]:
import pandas as pd
import os

# List of file paths
file_paths = [
    'yellow_tripdata_2019-01.csv',
    'yellow_tripdata_2019-02.csv',
    'yellow_tripdata_2019-03.csv',
    'yellow_tripdata_2019-04.csv',
    'yellow_tripdata_2019-05.csv',
    'yellow_tripdata_2019-06.csv',
    'yellow_tripdata_2019-07.csv',
    'yellow_tripdata_2019-08.csv',
    'yellow_tripdata_2019-09.csv',
    'yellow_tripdata_2019-10.csv',
    'yellow_tripdata_2019-11.csv',
    'yellow_tripdata_2019-12.csv'
]

# Initialize an empty list to hold the dataframes
df_list = []

# Read each file and append the dataframe to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatenate all dataframes in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Output file path
output_file = 'combined_yellow_tripdata_2019.csv'

# Write the combined dataframe to a new CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined file written to {output_file}")




Combined file written to combined_yellow_tripdata_2019.csv


In [ ]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
from modin.config import Engine
import time
import re
import gzip
import os

# Read using Pandas
start_time = time.time()
df_pandas = pd.read_csv('combined_yellow_tripdata_2019.csv')
pandas_time = time.time() - start_time
print(f"Pandas read time: {pandas_time} seconds")

# Read using Dask
start_time = time.time()
df_dask = dd.read_csv('combined_yellow_tripdata_2019.csv')
df_dask = df_dask.compute()
dask_time = time.time() - start_time
print(f"Dask read time: {dask_time} seconds")


# Ensure that Ray is the engine used by Modin
Engine.put("ray")

# Shutdown any existing Ray instances
ray.shutdown()

# Initialize Ray
ray.init()

# Timing the read_csv operation with Modin
start_time = time.time()
df_modin_ray = mpd.read_csv('combined_yellow_tripdata_2019.csv')
modin_ray_time = time.time() - start_time
print(f"Modin (Ray) read time: {modin_ray_time} seconds")

# Shutdown Ray after the operations
ray.shutdown()


# Clean column names
def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9]+', '_').str.strip()
    return df

df_pandas = clean_column_names(df_pandas)
df_dask = clean_column_names(df_dask)
df_modin_ray = clean_column_names(df_modin_ray)




Pandas read time: 1.37904953956604 seconds
Dask read time: 2.2482693195343018 seconds


2024-06-09 21:40:25,857	INFO worker.py:1753 -- Started a local Ray instance.


Modin (Ray) read time: 3.658999443054199 seconds


In [19]:
import yaml

schema = {
    'columns': [
        'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
        'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
        'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
        'total_amount', 'congestion_surcharge'
    ],
    'separator': ','
}

with open('schema.yaml', 'w') as file:
    yaml.dump(schema, file)

print("Schema written to schema.yaml")


Schema written to schema.yaml


'schema = {\n    \'columns\': [\n        {\'name\': \'VendorID\', \'type\': \'int64\'},\n        {\'name\': \'tpep_pickup_datetime\', \'type\': \'object\'},\n        {\'name\': \'tpep_dropoff_datetime\', \'type\': \'object\'},\n        {\'name\': \'passenger_count\', \'type\': \'float64\'},\n        {\'name\': \'trip_distance\', \'type\': \'float64\'},\n        {\'name\': \'RatecodeID\', \'type\': \'float64\'},\n        {\'name\': \'store_and_fwd_flag\', \'type\': \'object\'},\n        {\'name\': \'PULocationID\', \'type\': \'float64\'},\n        {\'name\': \'DOLocationID\', \'type\': \'float64\'},\n        {\'name\': \'payment_type\', \'type\': \'float64\'},\n        {\'name\': \'fare_amount\', \'type\': \'float64\'},\n        {\'name\': \'extra\', \'type\': \'float64\'},\n        {\'name\': \'mta_tax\', \'type\': \'float64\'},\n        {\'name\': \'tip_amount\', \'type\': \'float64\'},\n        {\'name\': \'tolls_amount\', \'type\': \'float64\'},\n        {\'name\': \'improvement_sur

In [20]:
# Load the schema
with open('schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

# Validate schema
def validate_schema(df, schema):
    expected_columns = schema['columns']
    if list(df.columns) != expected_columns:
        raise ValueError(f"Column names do not match the schema. Expected: {expected_columns}, Found: {list(df.columns)}")
    if len(df.columns) != len(expected_columns):
        raise ValueError(f"Number of columns do not match the schema. Expected: {len(expected_columns)}, Found: {len(df.columns)}")

validate_schema(df_pandas, schema)
validate_schema(df_dask, schema)
validate_schema(df_modin_ray, schema)

print("Validation successful!")

# Write to pipe-separated gzip file
def write_compressed_file(df, output_file):
    with gzip.open(output_file, 'wt', encoding='utf-8') as f:
        df.to_csv(f, sep='|', index=False)

output_file = 'output_file.txt.gz'
write_compressed_file(df_pandas, output_file)

print(f"File written to {output_file}")

# Generate summary
summary = {
    'Total number of rows': len(df_pandas),
    'Total number of columns': len(df_pandas.columns),
    'File size': os.path.getsize(output_file)
}

print("Summary of the file:")
for key, value in summary.items():
    print(f"{key}: {value}")

Validation successful!
File written to output_file.txt.gz
Summary of the file:
Total number of rows: 488572
Total number of columns: 18
File size: 8717406
